# Global Minimial Viable Products (1)
This is only for CONUS currently as the potential runoff product is to large to be an asset. 

Combine:
* SMAP available porosity
* ERA5-Merit potential runoff
* MERIT terrain slope
* MERIT upstream contributing area (needs to be less than some threshold)

In to various "products".

In [1]:
import ee
import numpy as np

import sys
sys.path.insert(1, '../datasets/SMAP-USDA/')  # until we package this
import visualize as vis
import water_common as common
import normalize as norm
from statistics import mean

Loading BokehJS ...

In [2]:
#ee.Authenticate()
ee.Initialize()

In [3]:
# Some common parameters
month_names = [
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
days_per_month = [
    31, 28, 31, 30, 31, 30,
    31, 31, 30, 31, 30, 31] 
seconds_per_month = [ dd* 24*60*60 for dd in days_per_month ]
month = 7
month_name = month_names[month - 1]

bbox = common.bboxes()['conus']

# SMAP-USDA Available Porosity
More available porosity is better.

In [4]:
# Get a single example image 
smap_usda_clim = ee.ImageCollection('users/jamesmcc/smap_usda_climatology')

avail_porosity_img = (
    smap_usda_clim
    .filter(ee.Filter.eq('month', month))
    .filter(ee.Filter.eq('band', 'avail_porosity_mm')).first())

avail_porosity_img_scaled = norm.img_scale(
    avail_porosity_img, area_of_interest=common.bboxes()['conus'])

In [5]:
# smap_range = norm.img_range(img, area_of_interest=common.bboxes()['conus'])
# smap_range

In [6]:
smap_missing = -99999

# units: mm
avail_porosity_data = np.array(
        avail_porosity_img
        .sampleRectangle(bbox, ['avail_porosity_mm'], smap_missing)
        .get('avail_porosity_mm')
        .getInfo())
avail_porosity_data_mask = np.ma.masked_values(avail_porosity_data, smap_missing)
avail_porosity_data_drop = avail_porosity_data_mask[~avail_porosity_data_mask.mask]

# Scaled
avail_porosity_scaled_data = np.array(
        avail_porosity_img_scaled
        .sampleRectangle(bbox, ['avail_porosity_mm'], smap_missing)
        .get('avail_porosity_mm')
        .getInfo())
avail_porosity_scaled_data_mask = np.ma.masked_values(avail_porosity_scaled_data, smap_missing)
avail_porosity_scaled_data_drop = avail_porosity_scaled_data_mask[~avail_porosity_scaled_data_mask.mask]

In [7]:
#print(avail_porosity_data)
#avail_porosity_data_mask
#avail_porosity_data_drop

In [8]:
_ = norm.normal_dist_plot(avail_porosity_data_drop)

In [9]:
_ = norm.normal_dist_plot(avail_porosity_scaled_data_drop)

## ERA5-MERIT Potential Runoff

In [10]:
# Get a single example image 
pot_runoff_clim = ee.ImageCollection(
    'users/jamesmcc/era5_merit_potential_sfc_runoff_conus_climatology')

pot_runoff_img = (
    pot_runoff_clim
    .filter(ee.Filter.eq('month', month))
    .filter(ee.Filter.eq('variable', 'potential_sfc_runoff_mon_clim_cms')).first())

# Mask to a usable range
runoff_mask = pot_runoff_img.lte(.5).And(pot_runoff_img.gte(.01))

pot_runoff_img_scaled = norm.img_scale(
    pot_runoff_img.updateMask(runoff_mask), 
    area_of_interest=common.bboxes()['conus'])

In [11]:
# Image.sampleRectangle: Too many pixels in sample; must be <= 262144. Got 2047606081.
# https://gis.stackexchange.com/questions/350771/earth-engine-simplest-way-to-move-from-ee-image-to-array-for-use-in-sklearn/351177#351177
# runoff_missing = -99999
# pot_runoff_data = np.array(
#         pot_runoff_img_scaled
#         .sampleRectangle(bbox, ['potential_sfc_runoff_mon_clim_cms'], smap_missing)
#         .get('potential_sfc_runoff_mon_clim_cms')
#         .getInfo())
# pot_runoff_data_mask = np.ma.masked_values(pot_runoff_data, smap_missing)
# pot_runoff_data_drop = pot_runoff_data_mask[~pot_runoff_data_mask.mask]

In [12]:
# print(pot_runoff_data)

In [13]:
# pot_runoff_data_mask
# pot_runoff_data_drop

In [14]:
# _ = norm.normal_dist_plot(avail_porosity_data_drop)

## MERIT Terrain Slope

In [15]:
slope_img = ee.Image('users/jamesmcc/merit_slope/merit_terrain_slope')
slope_mask = slope_img.lte(9).And(slope_img.gte(2))

In [16]:
# This dosent really need scaled, just a mask?
#slope_img_scaled = norm.img_scale(slope_img, area_of_interest=common.bboxes()['conus'])

## Product 1
Simply give higher scores to places that have both high potential runoff and available porosity in only areas with acceptable slopes

In [17]:
product_1 = avail_porosity_img_scaled.multiply(pot_runoff_img_scaled).updateMask(slope_mask).updateMask(runoff_mask)

In [18]:
product_1_range = norm.img_range(product_1, area_of_interest=bbox)

palette_name = 'BrBG'
palette_len = 11
palette = vis.brewer[palette_name][palette_len][::-1]
vis.legend(palette=palette, minimum=product_1_range[0], maximum=product_1_range[1])

box_corners = bbox.toGeoJSON()['coordinates'][0]
center_lon = mean([corner[0] for corner in box_corners])
center_lat = mean([corner[1] for corner in box_corners])


vis_params = {
    'min': product_1_range[0], 'max': product_1_range[1], 'dimensions': 512,
    'palette': palette}

In [19]:
the_map = vis.folium_map(location=[center_lat, center_lon], zoom_start=4, height=500)
month_names = [
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_name = month_names[month-1]

the_map.add_ee_layer(product_1, vis_params, 'Product 1 ' + month_name)

vis.folium_display(the_map)

## Product 2
Ratio of potential_runoff:available_porosity?

In [20]:
runoff_pixel_area_m2 = pot_runoff_img.pixelArea()
pot_runoff_img_mm = pot_runoff_img.divide(runoff_pixel_area_m2).multiply(seconds_per_month[month-1] * 1000)
# pot_runoff_mask = pot_runoff_img_mm.gte(1000)
avail_porosity_mask = avail_porosity_img.gte(50)

In [21]:
norm.img_range(pot_runoff_img_mm.updateMask(runoff_mask), area_of_interest=bbox)
#norm.img_range(pot_runoff_img_mm, area_of_interest=bbox)

[16.847965699571173, 1162.8066667861892]

In [22]:
norm.img_range(runoff_pixel_area_m2, area_of_interest=bbox)

[1147915.625, 1592437.625]

In [23]:
norm.img_range(avail_porosity_img, area_of_interest=bbox)

[0, 205.34410095214844]

In [24]:
product_2_raw = (
    pot_runoff_img_mm
    .divide(avail_porosity_img)
    .updateMask(slope_mask)
    .updateMask(avail_porosity_mask)
    .updateMask(runoff_mask))
product_2_mask = product_2_raw.gte(1)
product_2 = product_2_raw.updateMask(product_2_mask)

In [25]:
product_2_range = norm.img_range(product_2, area_of_interest=bbox)


palette_name = 'BrBG'
palette_len = 11
palette = vis.brewer[palette_name][palette_len][::-1]
vis.legend(palette=palette, minimum=product_2_range[0], maximum=product_2_range[1])

box_corners = bbox.toGeoJSON()['coordinates'][0]
center_lon = mean([corner[0] for corner in box_corners])
center_lat = mean([corner[1] for corner in box_corners])

vis_params = {
    'min': product_2_range[0], 'max': product_2_range[1], 'dimensions': 512,
    'palette': palette}

In [26]:
the_map = vis.folium_map(location=[center_lat, center_lon], zoom_start=4, height=500)
the_map.add_ee_layer(product_2, vis_params, 'Product 2: ' + month_name)
vis.folium_display(the_map)